In [1]:
# Użycie zapisanych na dysku zdjęć.
!unzip -q '/content/drive/My Drive/train_val_test.zip' -d '/content'
# Load the TensorBoard notebook extension. #%reload_ext tensorboard
%load_ext tensorboard

In [2]:
from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(2)

import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adamax, Nadam
from tensorflow.keras.regularizers import l1_l2

BATCH_SIZE = 32
EPOCHS = 50
INITIAL_EPOCH = 0
IMG_HEIGHT, IMG_WIDTH, CHANNELS = 220, 220, 3

dir_path = "/content/train_val_test"
root = "/content/drive/My Drive/CNN_xception/" 

train_gen = ImageDataGenerator(rescale=1.0/255.)
val_gen = ImageDataGenerator(rescale=1.0/255.)
test_gen = ImageDataGenerator(rescale=1.0/255.)

train_generator = train_gen.flow_from_directory(
    dir_path + "/train",
    batch_size=BATCH_SIZE,
    shuffle=True,
    color_mode="rgb",
    class_mode="categorical",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    seed=2)
validation_generator = val_gen.flow_from_directory(
    dir_path + "/val",
    batch_size=BATCH_SIZE,
    shuffle=True,
    color_mode="rgb",
    class_mode="categorical",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    seed=2)
test_generator = test_gen.flow_from_directory(
    dir_path + "/test",
    batch_size=1,
    shuffle=False,
    color_mode="rgb",
    class_mode="categorical",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    seed=2)

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n // validation_generator.batch_size

my_callbacks = [
    ModelCheckpoint(filepath=root + "model.{epoch:02d}.h5",
                    monitor="val_accuracy",
                    mode="max",
                    save_best_only=True,
                    save_freq="epoch",),
    EarlyStopping(monitor="val_loss",
                  patience=5,),
    TensorBoard(log_dir=root + "logs",
                write_images=False,
                histogram_freq=1,
                embeddings_freq=2,),
]

Found 20647 images belonging to 5 classes.
Found 6955 images belonging to 5 classes.
Found 6221 images belonging to 5 classes.


In [ ]:
# Użycie zapisanego na dysku modelu.
files = [f for f in sorted(os.listdir(root))]
model_file = files[-1]
INITIAL_EPOCH = int(model_file.split('.')[1])
EPOCHS += INITIAL_EPOCH
    
model = load_model(root + model_file)
print(f"Wczytanie pliku modelu: {model_file}, z ilością EPOCH: {INITIAL_EPOCH}")

Wczytanie pliku modelu: model.07.h5, z ilością EPOCH: 7


In [ ]:
# Wczytanie modelu, ustawienie wag.
base_model = applications.Xception(weights="imagenet", 
                                   include_top=False, 
                                   pooling="avg",
                                   input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))
# Zamrożenie części warstw by kożystać z wytrenowanych już wag.
trainable_layers = 100 # sieć ma jakoś 134 warstwy, 5 warst do wytrenowania to za mało
for i in range(len(base_model.layers) - trainable_layers):
    base_model.layers[i].trainable = False
    
# Dodanie warstwy do wczytanego modelu, uwzględnienie liczbę klas
x = base_model.output
predictions = Dense(5, activation="softmax")(x)
model = Model(inputs = base_model.input, outputs = predictions)

model.compile(optimizer=Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])
model.summary()

In [3]:
# Wczytanie modelu i ustawienie parametrów by nadawał się do problemu.
model = applications.Xception(weights=None, 
                                   include_top=True, 
                                   pooling="avg",
                                   classes=5,
                                   classifier_activation="softmax",
                                   input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))

model.compile(optimizer=Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])
model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 220, 220, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 109, 109, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 109, 109, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 109, 109, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
# Włączenie tensorboard.
%tensorboard --logdir "/content/drive/My Drive/CNN_xception/logs" 

In [4]:
history = model.fit(train_generator, 
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=EPOCHS, shuffle=True,
                    callbacks=my_callbacks,
                    initial_epoch=INITIAL_EPOCH,
                    use_multiprocessing=False, verbose=1)

Epoch 1/50
645/645 [==============================] - 470s 728ms/step - loss: 0.3387 - accuracy: 0.8757 - val_loss: 0.6112 - val_accuracy: 0.8436
Epoch 2/50
645/645 [==============================] - 464s 719ms/step - loss: 0.1567 - accuracy: 0.9461 - val_loss: 0.1921 - val_accuracy: 0.9453
Epoch 3/50
645/645 [==============================] - 467s 724ms/step - loss: 0.1121 - accuracy: 0.9613 - val_loss: 0.1624 - val_accuracy: 0.9506
Epoch 4/50
645/645 [==============================] - 465s 721ms/step - loss: 0.0886 - accuracy: 0.9694 - val_loss: 0.1437 - val_accuracy: 0.9551
Epoch 5/50
645/645 [==============================] - 466s 723ms/step - loss: 0.0750 - accuracy: 0.9744 - val_loss: 0.1202 - val_accuracy: 0.9664
Epoch 6/50
645/645 [==============================] - 464s 720ms/step - loss: 0.0618 - accuracy: 0.9790 - val_loss: 0.1587 - val_accuracy: 0.9529
Epoch 7/50
645/645 [==============================] - 465s 721ms/step - loss: 0.0515 - accuracy: 0.9833 - val_loss: 0.1284 -

In [5]:
# Sprawdzenie poprawności na danych testowych
test_generator.reset()
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size 
scores = model.evaluate(test_generator, 
                        steps=STEP_SIZE_TEST,
                        verbose=0)
print(model.metrics_names[0] + '=', scores[0])
print(model.metrics_names[1] + '=', scores[1])

test_generator.reset()
probabilities = model.predict(test_generator, 
                              steps=STEP_SIZE_TEST,
                              verbose=0)
y_test = probabilities.argmax(axis=1)
y_pred = test_generator.classes
target_names = sorted(os.listdir(dir_path + "/test"))
print(classification_report(y_test, y_pred, target_names=target_names))

loss= 0.12337103486061096
accuracy= 0.9678508043289185
              precision    recall  f1-score   support

      type_F       0.94      0.81      0.87       186
      type_N       0.98      0.96      0.97      2447
      type_Q       0.99      1.00      0.99      1590
      type_S       0.90      0.93      0.92       583
      type_V       0.96      0.98      0.97      1415

    accuracy                           0.97      6221
   macro avg       0.95      0.94      0.94      6221
weighted avg       0.97      0.97      0.97      6221



In [6]:
# Trenowanie na danych ze zbioru walidacyjnego.
model.fit(validation_generator, 
          steps_per_epoch=STEP_SIZE_VALID,
        #   validation_data=validation_generator,
        #   validation_steps=STEP_SIZE_VALID,
          epochs=2, shuffle=True,
          use_multiprocessing=False, verbose=1)

# Sprawdzenie poprawności na danych testowych po trenowaniu na zbiorze walidacyjnym.
test_generator.reset()
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size 
scores = model.evaluate(test_generator, 
                        steps=STEP_SIZE_TEST,
                        verbose=0)
print(model.metrics_names[0] + '=', scores[0])
print(model.metrics_names[1] + '=', scores[1])

test_generator.reset()
probabilities = model.predict(test_generator, 
                              steps=STEP_SIZE_TEST,
                              verbose=0)
y_test = probabilities.argmax(axis=1)
y_pred = test_generator.classes
target_names = sorted(os.listdir(dir_path + "/test"))
print(classification_report(y_test, y_pred, target_names=target_names))

Epoch 1/2
217/217 [==============================] - 144s 663ms/step - loss: 0.1006 - accuracy: 0.9682
Epoch 2/2
217/217 [==============================] - 143s 657ms/step - loss: 0.0439 - accuracy: 0.9856
loss= 0.10984040796756744
accuracy= 0.9680115580558777
              precision    recall  f1-score   support

      type_F       0.96      0.73      0.83       209
      type_N       0.95      0.98      0.97      2320
      type_Q       1.00      0.99      0.99      1623
      type_S       0.93      0.96      0.94       585
      type_V       0.98      0.96      0.97      1484

    accuracy                           0.97      6221
   macro avg       0.96      0.92      0.94      6221
weighted avg       0.97      0.97      0.97      6221



In [7]:
# Zapisanie pliku .csv porównując typ i predykcję, jest podane zdjęcie więc łatwo będzie sprawdzić co z nim nie tak
test_generator.reset()
probabilities = model.predict(test_generator, 
                              steps=STEP_SIZE_TEST,
                              verbose=1)
predicted_class_indices = np.argmax(probabilities, axis=1)
labels = (train_generator.class_indices)
labels = dict((v, k) for k, v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames = test_generator.filenames
results=pd.DataFrame({"Filename": filenames,
                      "Predictions": predictions,})
results.to_csv("results.csv", sep=';', index=False)

6221/6221 [==============================] - 54s 9ms/step


In [9]:
# Tworzenie schematu zawierającego kształt warstw.
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file="model.png")

TypeError: ignored